In [79]:
import numpy as np
import pandas as pd
from string import punctuation

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [80]:
df=pd.read_csv('data/ArticlesMay2017.csv', encoding='latin')
df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,5906b3197c459f24986dd003,2412,By IAN JOHNSON,article,My Beijing: The Sacred City,"['Travel and Vacations', 'Beijing (China)', 'R...",3,Travel,1,2017-05-01 04:01:21,Unknown,"This metropolis was once a total work of art, ...",The New York Times,News,https://www.nytimes.com/2017/05/01/travel/beij...
1,NaN,5906b3297c459f24986dd004,2318,By EMMA G. FITZSIMMONS,article,"6 Million Riders a Day, 1930s Technology","['Subways', 'Delays (Transportation)', 'Transi...",68,Metro,1,2017-05-01 04:01:33,Unknown,New Yorkâs subway is struggling with old inf...,The New York Times,News,https://www.nytimes.com/2017/05/01/nyregion/ne...
2,NaN,5906ceec7c459f24986dd021,1795,By MARC TRACY,article,Seeking a Cross-Border Conference,"['Cetys University', 'College Athletics', 'Nat...",3,Sports,1,2017-05-01 06:00:05,Unknown,Cetys University is making an ambitious bid to...,The New York Times,News,https://www.nytimes.com/2017/05/01/sports/mexi...
3,NaN,5906cfa77c459f24986dd022,213,By SHANNON DOYNE,article,"Questions for: âDespite the âYuck Factor,â...",[],3,Learning,0,2017-05-01 06:03:03,Unknown,How are leeches used to treat various medical ...,The New York Times,News,https://www.nytimes.com/2017/05/01/learning/qu...
4,NaN,5906e1c07c459f24986dd039,1342,By JASON STANLEY,article,Who Is a âCriminalâ?,"['Illegal Immigration', 'Traffic and Parking V...",3,OpEd,0,2017-05-01 07:20:26,Unknown,Justice Roberts was right. The Trump administr...,The New York Times,Op-Ed,https://www.nytimes.com/2017/05/01/opinion/who...


In [81]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  16
Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [82]:
# headline 열의 데이터만 사용, Null 검사
df['headline'].isnull().values.any()

False

In [83]:
# headline 열의 데이터로 리스트 생성
headline = [title for title in df.headline.values]
headline[:5]

['My Beijing: The Sacred City',
 '6 Million Riders a Day, 1930s Technology',
 'Seeking a Cross-Border Conference',
 'Questions for: â\x80\x98Despite the â\x80\x9cYuck Factor,â\x80\x9d Leeches Are Big in Russian Medicineâ\x80\x99',
 'Who Is a â\x80\x98Criminalâ\x80\x99?']

In [84]:
len(headline)

996

In [85]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

935

In [86]:
# 구두점 제거와 소문자화를 위한 함수
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii", 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [87]:
text = [repreprocessing(x) for x in headline]
text[:5]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal']

In [110]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 2653


In [89]:
print(t.word_index)

{'the': 1, 'a': 2, 'to': 3, 'of': 4, 'in': 5, 'for': 6, 'and': 7, 'trump': 8, 'is': 9, 'on': 10, 'new': 11, 'how': 12, 'with': 13, 'you': 14, 'at': 15, 'an': 16, 'season': 17, 'what': 18, 'it': 19, 'its': 20, 'york': 21, 'episode': 22, 'trumps': 23, 'from': 24, 'as': 25, 'be': 26, 'not': 27, 'are': 28, 'may': 29, 'when': 30, 'but': 31, 'after': 32, 'about': 33, '3': 34, 'today': 35, 'this': 36, 'us': 37, 'recap': 38, 'by': 39, 'we': 40, 'comey': 41, 'house': 42, 'questions': 43, 'health': 44, 'president': 45, 'bill': 46, 'that': 47, 'my': 48, 'too': 49, 'get': 50, 'was': 51, 'life': 52, 'or': 53, 'will': 54, 'why': 55, 'your': 56, 'city': 57, 'dont': 58, 'care': 59, 'more': 60, 'do': 61, 'can': 62, 'has': 63, 'no': 64, 'up': 65, 'time': 66, 'out': 67, 'into': 68, 'better': 69, 'war': 70, 'his': 71, 'their': 72, 'says': 73, 'american': 74, 'love': 75, 'one': 76, 'gop': 77, '5': 78, 'climate': 79, 'should': 80, 'donald': 81, 'getting': 82, 'mr': 83, 'james': 84, 'like': 85, 'first': 86, 

In [90]:
print(t.word_index['i'])

138


In [91]:
text.sort()
print(text)

['  two red bull teams in champions league rules say ', '100 days', '11 questions to ask before getting a divorce', '1908  a harrowing maritime rescue told from the high seas', '4yearolds dont act like this', '6 million riders a day 1930s technology', '9 letters starts with c ends with d', 'a 110 billion weapons sale to saudi arabia has kushners personal touch', 'a 500 rental deposit in the city its a deal', 'a bestiary of the mind', 'a bettertasting bird', 'a call to action winners from our fourth annual student editorial contest', 'a christian abortion doctor', 'a college scourge of plagiarized language this time in code', 'a common word but not a common name', 'a coop renovation project is frustrating to neighbors', 'a cry for help as a witness recounts a mans fall onto the subway tracks', 'a disaster wrapped in victory', 'a disgraceful exit from the paris pact', 'a doctors unexpected call', 'a driveable feast', 'a fantastic fantastical book for the new cool woman in my life', 'a fe

In [111]:
print(text[386])

i accidentally killed a child may i contact the family


In [93]:
sequences = []
for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[106, 206],
 [106, 206, 975],
 [106, 206, 975, 501],
 [106, 206, 975, 501, 5],
 [106, 206, 975, 501, 5, 976],
 [106, 206, 975, 501, 5, 976, 502],
 [106, 206, 975, 501, 5, 976, 502, 292],
 [106, 206, 975, 501, 5, 976, 502, 292, 116],
 [269, 438],
 [710, 43],
 [710, 43, 3]]

In [94]:
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 :', index_to_word[1])
print('빈도수 상위 582번 단어 :', index_to_word[582])

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : taiwan


In [95]:
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 16


In [96]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0 106 206]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 106 206 975]
 [  0   0   0   0   0   0   0   0   0   0   0   0 106 206 975 501]]


In [97]:
x = sequences[:,:-1]
y = sequences[:,-1]

In [98]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [99]:
x.shape, y.shape

((5501, 15), (5501, 2653))

## 2. 모델 설계 및 학습

In [100]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [101]:
# 임베딩 벡터는 10차원, 은닉 상태 크기는 128
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, 
                    name="Embedding_Layer"))
model.add(LSTM(128, name="LSTM_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 15, 10)            26530     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 2653)              342237    
Total params: 439,935
Trainable params: 439,935
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [103]:
history = model.fit(x, y, epochs=200, verbose=2)

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
 - 2s - loss: 7.4983 - accuracy: 0.0334
Epoch 2/200
 - 2s - loss: 7.0002 - accuracy: 0.0369
Epoch 3/200
 - 2s - loss: 6.8756 - accuracy: 0.0369
Epoch 4/200
 - 2s - loss: 6.7753 - accuracy: 0.0396
Epoch 5/200
 - 2s - loss: 6.6665 - accuracy: 0.0474
Epoch 6/200
 - 2s - loss: 6.5402 - accuracy: 0.0484
Epoch 7/200
 - 2s - loss: 6.3966 - accuracy: 0.0509
Epoch 8/200
 - 2s - loss: 6.2294 - accuracy: 0.0544
Epoch 9/200
 - 2s - loss: 6.0528 - accuracy: 0.0571
Epoch 10/200
 - 2s - loss: 5.8736 - accuracy: 0.0627
Epoch 11/200
 - 2s - loss: 5.7039 - accuracy: 0.0682
Epoch 12/200
 - 2s - loss: 5.5427 - accuracy: 0.0767
Epoch 13/200
 - 2s - loss: 5.3862 - accuracy: 0.0796
Epoch 14/200
 - 2s - loss: 5.2344 - accuracy: 0.0893
Epoch 15/200
 - 2s - loss: 5.0896 - accuracy: 0.0938
Epoch 16/200
 - 2s - loss: 4.9486 - accuracy: 0.0983
Epoch 17/200
 - 2s - loss: 4.8140 - accuracy: 0.1103
Epoch 18/200
 - 2s - loss: 4.6844 - accuracy: 0.1223
Epoch 19/200
 - 2s - loss: 4.5549 - accuracy: 0.1383
Ep

Epoch 155/200
 - 2s - loss: 0.3826 - accuracy: 0.8940
Epoch 156/200
 - 2s - loss: 0.3797 - accuracy: 0.8953
Epoch 157/200
 - 2s - loss: 0.3769 - accuracy: 0.8951
Epoch 158/200
 - 2s - loss: 0.3740 - accuracy: 0.8931
Epoch 159/200
 - 2s - loss: 0.3719 - accuracy: 0.8955
Epoch 160/200
 - 2s - loss: 0.3703 - accuracy: 0.8964
Epoch 161/200
 - 2s - loss: 0.3675 - accuracy: 0.8975
Epoch 162/200
 - 2s - loss: 0.3661 - accuracy: 0.8964
Epoch 163/200
 - 2s - loss: 0.3659 - accuracy: 0.8960
Epoch 164/200
 - 2s - loss: 0.3699 - accuracy: 0.8933
Epoch 165/200
 - 2s - loss: 0.3853 - accuracy: 0.8889
Epoch 166/200
 - 2s - loss: 0.3789 - accuracy: 0.8913
Epoch 167/200
 - 2s - loss: 0.3615 - accuracy: 0.8935
Epoch 168/200
 - 2s - loss: 0.3559 - accuracy: 0.8953
Epoch 169/200
 - 2s - loss: 0.3534 - accuracy: 0.8957
Epoch 170/200
 - 2s - loss: 0.3522 - accuracy: 0.8942
Epoch 171/200
 - 2s - loss: 0.3492 - accuracy: 0.8977
Epoch 172/200
 - 2s - loss: 0.3498 - accuracy: 0.8955
Epoch 173/200
 - 2s - loss: 

In [104]:
## 3. 모델 검증

In [126]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=15, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [129]:
print(sentence_generation(model, t, 'i', 9))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i defuse cyberattack travel the 500 summer in the opponents
